In [3]:
pip install gym

In [5]:
import gym
env = gym.make("Taxi-v3").env

In [6]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [12]:
help (gym.make)

Help on function make in module gym.envs.registration:

make(id, **kwargs)



In [15]:
!pip install mujoco_py

     |████████████████████████████████| 798kB 2.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Using cached https://files.pythonhosted.org/packages/a2/eb/0e2f9bb76ff799dde0c6c078044506be141ef9442fce3e7b55c5fa2aec26/fasteners-0.16.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/13/d7/79c091c877493de7f8286ed62c77bf0f2c51105656073846b2326021b524/glfw-2.1.0-py2.py27.py3.py30.py31.py32.py33.py34.py35.py36.py37.py38-none-manylinux2014_x86_64.whl
  ERROR: Failed building wheel for mujoco-py
  Running setup.py clean for mujoco-py
Failed to build mujoco-py
ERROR: Could not build wheels for mujoco-py which use PEP 517 and cannot be installed directly


In [16]:
import gym
env = gym.make('Hopper-v2').env
env.render()


DependencyNotInstalled: ignored

In [42]:
import gym
env = gym.make('CartPole-v0')
env.reset()
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample()) # take a random action
env.close()

In [11]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [19]:
state = env.encode(2, 2, 0, 0) # (taxi row, taxi column,  destination index, passenger index)
print("State:", state)
env.s = state
env.render()

State: 240
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [23]:
env.P[241]

{0: [(1.0, 341, -1, False)],
 1: [(1.0, 141, -1, False)],
 2: [(1.0, 261, -1, False)],
 3: [(1.0, 221, -1, False)],
 4: [(1.0, 241, -10, False)],
 5: [(1.0, 241, -10, False)]}

In [21]:
env.P[240]

{0: [(1.0, 340, -1, False)],
 1: [(1.0, 140, -1, False)],
 2: [(1.0, 260, -1, False)],
 3: [(1.0, 220, -1, False)],
 4: [(1.0, 240, -10, False)],
 5: [(1.0, 240, -10, False)]}

In [38]:
env.s = 240  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))


Timesteps taken: 563
Penalties incurred: 168


In [40]:
len(frames)

563

In [42]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 563
State: 0
Action: 5
Reward: 20


In [44]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [47]:
q_table.shape

(500, 6)

In [48]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 1min 9s, sys: 16 s, total: 1min 25s
Wall time: 1min 12s


In [51]:
import pandas as pd
pd.DataFrame(q_table)

,0,1,2,3,4,5
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-2.418371,-2.363951,-2.418371,-2.363951,-2.273252,-11.363950
2,-1.870144,-1.450240,-1.870144,-1.450240,-0.750400,-10.450240
3,-2.363951,-2.273252,-2.363951,-2.273252,-2.122086,-11.273251
4,-2.496192,-2.496978,-2.496192,-2.496563,-9.865518,-9.697469
...,...,...,...,...,...,...
495,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
496,-2.157869,-2.122041,-2.145448,-2.122044,-5.758770,-5.132992
497,-0.994180,0.415999,-0.998105,-1.270423,-4.203360,-4.708095
498,-2.165447,-2.122079,-2.169741,-2.122080,-3.706797,-5.755925


In [61]:
np.argmax(q_table[452]), q_table[452]

(1, array([-2.47677047, -2.47061344, -2.47774308, -2.48388652, -9.6777776 ,
        -9.29260983]))

In [52]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 10

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1
        clear_output(wait=True)
        env.render()
        sleep(0.2)

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
Results after 10 episodes:
Average timesteps per episode: 14.1
Average penalties per episode: 0.0


In [ ]:
q_table[100]

array([0., 0., 0., 0., 0., 0.])